In [11]:
##indexing##
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
loader=WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/"),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        ),
    )
)

docs=loader.load()

#split
text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300,chunk_overlap=50)
splits=text_splitter.split_documents(docs) #->list

#embed
embedding_model=HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh-v1.5")
vectorstore=Chroma.from_documents(documents=splits,embedding=embedding_model)
retriever=vectorstore.as_retriever()

#ollama llm
llm=ChatOllama(model="llama3.2:3b-instruct-q8_0",temperature=0)

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Multi query

template="""You are an AI language model assistant. Your task is to generate three
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""

prompt_Multiquery=ChatPromptTemplate.from_template(template)

generate_queries=(
    prompt_Multiquery
    | llm
    | StrOutputParser()
    | (lambda x : x.split("\n")) #->["query1","query2"...]
)


retriever.map(): 接收多個 query <br>
對每個 query 去 vectorstore 找相關的 Document <br>
回傳 list[list[Document]]，每個 query 對應一個 Document 列表 例如: <br>

[ <br>
  [doc1, doc2, doc3],   # query1 的結果 <br>
  [doc4, doc5],          # query2 的結果 <br>
  [doc6, doc7]           # query3 的結果 <br>
] <br>





In [13]:
from langchain_core.load import dumps,loads

#把前面得到的query重複的地方去掉並且整合
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    #dumps:把 Document 轉成字串
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"

retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

4

In [14]:
for doc in docs:
    print("Content:", doc.page_content)
    print("-" * 50)  # 分隔線


Content: Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.
Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simp

In [ ]:
from operator import itemgetter


# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)



final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'According to the text, task decomposition can be done in three ways for LLM (Large Language Model) agents:\n\n1. With simple prompting, such as "Steps for XYZ." or "What are the subgoals for achieving XYZ?"\n2. Using task-specific instructions, e.g., "Write a story outline."\n3. With human inputs.\n\nThis process involves breaking down complex tasks into smaller and simpler steps, allowing the LLM agent to plan ahead and utilize more test-time computation.'